In [1]:
ifn='../../data/dists/data.top_words.combined.csv'

In [2]:
!pwd

/home/ryan/github/abslithist/notebooks/dists


In [3]:
import sys; sys.path.append('../..')
from abslithist import *
from abslithist.words import *

In [37]:
fields=get_allfields()
fieldsnow = {'abs':fields['Abs-Conc.Median.Abs.median'], 'conc':fields['Abs-Conc.Median.Conc.median']}

def getabsconc(w):
    for k,v in fieldsnow.items():
        if w in v:
            return k
    return 'neither'

In [4]:
df=pd.read_csv(ifn)
df

,word2,count,mean,rank,word1,absconc,period_start
0,burlero,2,0.980984,1,a-la,Conc,1600
1,lilli,2,0.933669,2,a-la,Conc,1600
2,lero,2,0.926497,3,a-la,Conc,1600
3,bullen,2,0.870457,4,a-la,Conc,1600
4,tol,2,0.759921,5,a-la,Conc,1600
...,...,...,...,...,...,...,...
1206035,paprika,2,0.908866,17,zucchini,Conc,1900
1206036,tofu,2,0.907589,18,zucchini,Conc,1900
1206037,broccoli,2,0.904036,19,zucchini,Conc,1900
1206038,tortillas,2,0.903416,20,zucchini,Conc,1900


In [5]:
hist_avg=df.groupby(['absconc','word1','word2']).mean().reset_index()
hist_avg

,absconc,word1,word2,count,mean,rank,period_start
0,Abs,a.n.c,anti-fascist,2.0,0.851020,5.0,1900.0
1,Abs,a.n.c,gaullists,2.0,0.852178,4.0,1900.0
2,Abs,a.n.c,gemayel,2.0,0.843606,7.0,1900.0
3,Abs,a.n.c,human-rights,4.0,0.842640,1.0,1900.0
4,Abs,a.n.c,insurgency,2.0,0.850155,6.0,1900.0
...,...,...,...,...,...,...,...
1080879,Conc,λ,ŏ,2.0,0.792834,14.0,1700.0
1080880,Conc,λ,α,4.0,0.800296,1.0,1700.0
1080881,Conc,λ,γ,2.0,0.788646,17.0,1700.0
1080882,Conc,λ,ε,2.0,0.791066,15.0,1700.0


In [57]:
import lltk
C=lltk.load('CanonFiction')
mfw=list(C.mfw())
mfwn=25000
okwords=set(mfw[:mfwn])

In [58]:
hist_avg_nice = hist_avg[hist_avg.word1.isin(okwords) & hist_avg.word2.isin(okwords)]
hist_avg_nice

,absconc,word1,word2,count,mean,rank,period_start
16,Abs,abandon,abandoned,6.75,0.686333,5.75,1650.0
17,Abs,abandon,abandoning,8.75,0.685898,4.00,1650.0
21,Abs,abandon,accept,3.00,0.682627,7.00,1900.0
22,Abs,abandon,acquiesce,3.00,0.687366,6.00,1900.0
23,Abs,abandon,adopt,5.00,0.658414,6.00,1800.0
...,...,...,...,...,...,...,...
1079723,Conc,zigzag,rivulet,3.00,0.829060,5.00,1900.0
1079727,Conc,zigzag,serpentine,2.00,0.798103,18.00,1850.0
1079731,Conc,zigzag,steeply,2.00,0.828490,15.00,1800.0
1079732,Conc,zigzag,threading,2.00,0.819863,11.00,1900.0


In [69]:
hist_avg_nice.sort_values('rank')

,absconc,word1,word2,count,mean,rank,period_start
487966,Abs,unexplained,inexplicable,8.0,0.724721,1.0,1800.0
481534,Abs,unattainable,attainable,8.0,0.799390,1.0,1700.0
922283,Conc,rows,rafters,5.0,0.759132,1.0,1600.0
202548,Abs,fears,dreads,10.0,0.803155,1.0,1500.0
481574,Abs,unattainable,unchangeable,4.0,0.793428,1.0,1900.0
...,...,...,...,...,...,...,...
1009496,Conc,tank,gas,2.0,0.686648,27.0,1900.0
637397,Conc,crack,pout,2.0,0.713849,27.0,1700.0
542097,Conc,bananas,carrots,2.0,0.817605,28.0,1900.0
480183,Abs,ultimate,relatively,2.0,0.664431,28.0,1700.0


In [92]:
def df2g(df,max_rank=2):
    import networkx as nx
    G=nx.Graph()
    dfq=df[df['rank']<=max_rank]
    for i,row in tqdm(dfq.iterrows(),total=len(dfq)):
        for w in [row.word1,row.word2]:
            if not G.has_node(w):
                G.add_node(w,absconc=getabsconc(w))
        G.add_edge(row.word1,row.word2,weight=row['count'] + row['mean'], rank=row['rank'])
    return G



In [93]:
G = df2g(hist_avg_nice)
G_abs = df2g(hist_avg_nice[hist_avg_nice.absconc=='Abs'])
G_conc = df2g(hist_avg_nice[hist_avg_nice.absconc=='Conc'])

100%|██████████| 7767/7767 [00:00<00:00, 10652.12it/s]


In [94]:
G_abs.order(), G_abs.size()

(5727, 7075)

In [95]:
G_conc.order(), G_conc.size()

(5161, 7192)

In [96]:
# draw_bokeh(G,show_labels=False)

In [97]:
import networkx as nx
nx.write_gexf(G,ifn.replace('.csv',f'.graph.mfw{mfwn}.gexf'))
nx.write_gexf(G_abs,ifn.replace('.csv',f'.graph.mfw{mfwn}.abs.gexf'))
nx.write_gexf(G_conc,ifn.replace('.csv',f'.graph.mfw{mfwn}.conc.gexf'))